# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,299.19,73,20,7.20,MG,1708451850
1,1,udachny,66.4167,112.4000,247.93,99,96,2.07,RU,1708451850
2,2,ushuaia,-54.8000,-68.3000,284.96,58,40,9.26,AR,1708451828
3,3,lebu,-37.6167,-73.6500,289.83,84,97,3.38,CL,1708451850
4,4,blackmans bay,-43.0167,147.3167,287.89,87,3,3.11,AU,1708451851
5,5,ribeira grande,38.5167,-28.7000,291.37,72,75,6.26,PT,1708451851
6,6,raduzhny,62.0961,77.4750,257.05,89,59,6.34,RU,1708451851
7,7,howard springs,-12.4970,131.0470,301.10,95,96,2.18,AU,1708451851
8,8,yellowknife,62.4560,-114.3525,253.94,81,75,3.60,CA,1708451851
9,9,port mathurin,-19.6833,63.4167,299.62,67,94,10.69,MU,1708451851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
h_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
h_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
! pip install geoviews

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrowing to wind speed and humidity relation (50) and wind speed 10
Best_weath= city_data_df.loc[(city_data_df['Humidity']<=50) & (city_data_df['Wind Speed']<=10)]

# Drop any rows with null values
np.where(pd.isnull(Best_weath))
np.where(pd.isna(Best_weath))

# Display sample data
Best_weath.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,campo grande,-20.4428,-54.6464,303.90,48,75,5.66,BR,1708451756
17,17,al manshah,26.4769,31.8035,292.29,32,0,3.60,EG,1708451853
24,24,terenos,-20.4422,-54.8603,304.99,48,75,5.66,BR,1708451854
29,29,porto recanati,43.4365,13.6613,284.33,43,5,3.12,IT,1708451855
31,31,semera,11.5010,41.2010,303.44,39,11,2.36,ET,1708451855


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Best_weath[['City', 'Country','Lat','Lng','Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,campo grande,BR,-20.4428,-54.6464,48,
17,al manshah,EG,26.4769,31.8035,32,
24,terenos,BR,-20.4422,-54.8603,48,
29,porto recanati,IT,43.4365,13.6613,43,
31,semera,ET,11.5010,41.2010,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}
  
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
campo grande - nearest hotel: Hotel Metropolitan
al manshah - nearest hotel: No hotel found
terenos - nearest hotel: No hotel found
porto recanati - nearest hotel: Hotel Ristorante Bianchi Nicola
semera - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
carlisle - nearest hotel: IHG Army Hotels Carlisle Barracks - Historia
gaspe - nearest hotel: Hotel Plante
codrington - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
ubari - nearest hotel: فندق اوباري
general villegas - nearest hotel: Rucalén
bahia de kino - nearest hotel: Hotel Hacienda
marsh harbour - nearest hotel: Lofty Fig Villas
nouadhibou - nearest hotel: Hotel valencia
albany - nearest hotel: No hotel found
panama city beach - nearest hotel: Shores of Panama
tilisarao - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
kaiama - 

,City,Country,Lat,Lng,Humidity,Hotel Name
16,campo grande,BR,-20.4428,-54.6464,48,Hotel Metropolitan
17,al manshah,EG,26.4769,31.8035,32,No hotel found
24,terenos,BR,-20.4422,-54.8603,48,No hotel found
29,porto recanati,IT,43.4365,13.6613,43,Hotel Ristorante Bianchi Nicola
31,semera,ET,11.5010,41.2010,39,No hotel found
...,...,...,...,...,...,...
523,filadelfia,US,39.9523,-75.1638,38,"The Ritz-Carlton, Philadelphia"
526,sabha,LY,27.0377,14.4283,30,فندق الكاظم-سبها
529,pendra,IN,22.7667,81.9500,32,No hotel found
531,qamdo,CN,31.1667,97.2333,50,银杏商务宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["City", "Country", "Hotel Name"] )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)